In [18]:
import tensorflow as tf
import json, os, sys, time
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import anchor, losses, manage_checkpoint

In [19]:
config = {
    "exp_desc" : "perfection", #실험명
    "sub_desc" : "exp1", #하위실험명
    "image_dir" : "/home/dan/Desktop/images", #이미지경로
    "coco_api" : "/home/dan/prj/PythonAPI",
    "annotation_dir" : "/home/dan/Desktop/annotations", #어노테이션 경로
    "label_set" : ["head","helmet"], # 분류할 오브젝트 집합
    "input_shape" : [300, 300, 3], #모델입력영상 크기
    "arch" : "ssd96_v1",
    "backbone" : "mobilenet",
    "anchor_param" : {"ratios": [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
                           "scales": [0.1, 0.2, 0.375, 0.55, 0.725, 0.9, 1.075],
                           "fm_sizes": [38, 19, 10, 5, 3, 1],
                           "image_size": 300}, #anchor parameters
    "ckpt":
    {
        "save_type" : "best",
        "max_to_keep" : 10,
        "pretrained_type" : "init",
        "model_path" : None
    },
    "inference_mode" : "train", # train or mAP
    "train" :
    {
        "num_examples" : -1,
        "batch_size" : 16,
        "augmentation" : True,
        "random_crop_rate" : 0.3,
        "neg_ratio" : 3,
        "initial_lr" : 1e-3,
        "momentum" : 0.9,
        "weight_decay" : 5e-5,
        "num_epochs" : 300
    },
    "val":
    {
        "num_examples" : -1,
        "batch_size" : 64
    },
    "test":
    {
        "num_examples" : -1,
        "batch_size" : 1
    }

}
num_classes = len(config['label_set'])
ckpt_dir = os.path.join('ckpts', 'perfection', 'exp1')
log_dir = os.path.join('logs', 'perfection', 'exp1')
# if os.path.isdir(ckpt_dir):
#     raise ValueError("checkpoint directory exists. checkout your experiment name in configure file.")
# if os.path.isdir(log_dir):
#     raise ValueError("log directory exists. checkout your experiment name in configure file.")


# Dataset load

In [20]:
from coco import Dataset

In [21]:
sys.path.append(config['coco_api'])
from pycocotools.coco import COCO

ds_obj = Dataset(config, COCO)

In [22]:
batch_generator, train_length = ds_obj.load_data_generator('train', config)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [23]:
for i, (_, imgs, gt_confs, gt_locs) in enumerate(batch_generator.take(1)):
    print(imgs.shape, gt_confs.shape, gt_locs.shape)

(16, 300, 300, 3) (16, 8732) (16, 8732, 4)


# Model load

In [4]:
model = tf.keras.models.load_model("ssd300-mobilenet")

In [5]:
model.layers[1].trainable = False
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
functional_1 (Functional)       [(None, 38, 38, 192) 616256      input_3[0][0]                    
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, None, None, 5 569344      functional_1[0][1]               
__________________________________________________________________________________________________
sequential_4 (Sequential)       (None, None, None, 5 536576      sequential_3[0][0]               
_______________________________________________________________________________________

# Training loop

In [ ]:
@tf.function
def train_step(imgs, gt_confs, gt_locs, ssd, criterion, optimizer, config):
    with tf.GradientTape() as tape:
        confs, locs = ssd(imgs)

        conf_loss, loc_loss = criterion(
            confs, locs, gt_confs, gt_locs)

        loss = conf_loss + loc_loss
        l2_loss = [tf.nn.l2_loss(t) for t in ssd.trainable_variables]
        l2_loss = config['train']['weight_decay'] * tf.math.reduce_sum(l2_loss)
        loss += l2_loss

    gradients = tape.gradient(loss, ssd.trainable_variables)
    optimizer.apply_gradients(zip(gradients, ssd.trainable_variables))

    return loss, conf_loss, loc_loss, l2_loss

criterion = losses.create_losses(config['train']['neg_ratio'], num_classes)

In [ ]:
summary_writer = tf.summary.create_file_writer(log_dir)

t_loss = tf.metrics.Mean(name='train_loss')
t_conf_loss = tf.metrics.Mean(name='train_conf_loss')
t_loc_loss = tf.metrics.Mean(name='train_loc_loss')
# v_loss = tf.metrics.Mean(name='val_loss')
# v_conf_loss = tf.metrics.Mean(name='val_conf_loss')
# v_loc_loss = tf.metrics.Mean(name='val_loc_loss')

learning_rate = 1E-3
EPOCH = 50
START = 0
optimizer = tf.keras.optimizers.Adam(learning_rate)
for epoch in range(START+1, START+EPOCH+1):
    start = time.time()
    for i, (_, imgs, gt_confs, gt_locs) in enumerate(batch_generator):
        imgs = preprocess_input(imgs)
        loss, conf_loss, loc_loss, l2_loss = train_step(
            imgs, gt_confs, gt_locs, model, criterion, optimizer, config)
        t_loss(loss)
        t_conf_loss(conf_loss)
        t_loc_loss(loc_loss)
        
        print("Epoch {} iteration {} loss : {}".format(epoch, i, t_loss.result()))
    
    '''
    you can add the validation part
    
    for i, (_, imgs, gt_confs, gt_locs) in enumerate(val_generator):
        imgs = preprocess_input(imgs)
        val_confs, val_locs = model(imgs)
        val_conf_loss, val_loc_loss = criterion(val_confs, val_locs, gt_confs, gt_locs)
        v_loss(val_conf_loss+val_loc_loss)
        v_conf_loss(val_conf_loss)
        v_loc_loss(val_loc_loss)
    '''
    # Save checkpoint with a strategy.
    if epoch%10 == 0:
        model.save("{}.h5".format(epoch))

    with summary_writer.as_default():
        tf.summary.scalar('train_loss', t_loss.result(), step=epoch)
        tf.summary.scalar('train_conf_loss', t_conf_loss.result(), step=epoch)
        tf.summary.scalar('train_loc_loss', t_loc_loss.result(), step=epoch)
        #tf.summary.scalar('val_loss', v_loss.result(), step=epoch)
        #tf.summary.scalar('val_conf_loss', v_conf_loss.result(), step=epoch)
        #tf.summary.scalar('val_loc_loss', v_loc_loss.result(), step=epoch)
        tf.summary.scalar('learning_rate', learning_rate, step=epoch)
        
    t_loss.reset_states()
    t_conf_loss.reset_states()
    t_loc_loss.reset_states()
    #v_loss.reset_states()
    #v_conf_loss.reset_states()
    #v_loc_loss.reset_states()   